In [13]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import datasets
from tensorflow.keras.layers import Dense 
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical

In [2]:
# train_df = pd.read_csv("train.csv")
# test_df = pd.read_csv("test.csv")
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [3]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [4]:
X_train = train_images.reshape((50000,-1))
print(X_train.shape)
X_test = test_images.reshape((10000,-1))
print(X_test.shape)

(50000, 3072)
(10000, 3072)


In [5]:
y_train = train_labels.flatten()
y_test = test_labels.flatten()

In [6]:
def relu(x):
    return np.maximum(0,x)

def relu_derivative(x):
    return (x>0).astype(float)

def softmax(z):
    exp_z = np.exp(z- np.max(z,axis=1, keepdims=True))
    return exp_z/np.sum(exp_z, axis=1, keepdims=True)

def cross_entropy_loss(y,y_pred):
    m=y.shape[0]
    loss = - np.sum(y * np.log(y_pred+1e-8)) / m
    return loss

def weights_back_pro(X_train, y_train, iterations, learning_rate=0.01):
    m,n = X_train.shape
    layer2=32
    weights_12 = np.random.randn(n,layer2) * 0.01
    bias_12 = np.zeros((1,layer2))

    layer3=32
    weights_23 = np.random.randn(layer2, layer3) * 0.01
    bias_23 = np.zeros((1,layer3))

    output=10
    weights_3o = np.random.randn(layer3, output) * 0.01
    bias_3o = np.zeros((1,output))

    for i in range(iterations):
        layer1_O = relu(np.dot(X_train, weights_12) + bias_12)
        layer2_O = relu(np.dot(layer1_O, weights_23) + bias_23)
        predictions = softmax(np.dot(layer2_O, weights_3o)+ bias_3o)
        y_one_hot = np.zeros((m, output))
        y_one_hot[np.arange(m), y_train] = 1
        loss = cross_entropy_loss(y_one_hot, predictions)

        print(f"iteration{i}: Loss: {loss}")
        
        weights_3o_grad = (1/m) * np.dot(layer2_O.T, (predictions-y_one_hot))
        bias_3o_grad = (1/m) * np.sum(predictions-y_one_hot, axis=0, keepdims=True)
        error_2 = np.dot((predictions - y_one_hot), weights_3o.T) * relu_derivative(layer2_O)

        weights_23_grad = (1/m) * np.dot(layer1_O.T, error_2)
        bias_23_grad = (1/m) * np.sum(error_2, axis=0, keepdims=True)
        error_1 = np.dot(error_2, weights_23.T) * relu_derivative(layer1_O)

        weights_12_grad = (1/m) * np.dot(X_train.T, error_1)
        bias_12_grad = (1/m) * np.sum(error_1, axis=0, keepdims=True)

        weights_3o -= learning_rate * weights_3o_grad
        bias_3o -= learning_rate * bias_3o_grad
        
        weights_23 -= learning_rate * weights_23_grad
        bias_23 -= learning_rate * bias_23_grad
        
        weights_12 -= learning_rate * weights_12_grad
        bias_12 -= learning_rate * bias_12_grad
    return weights_12, bias_12, weights_23, bias_23, weights_3o, bias_3o

def predict(X, weights_1, biases_1, weights_2, biases_2, weights_3, biases_3):
    layer_output_1 = relu(np.dot(X, weights_1) + biases_1)
    layer_output_2 = relu(np.dot(layer_output_1, weights_2) + biases_2)
    predictions = softmax(np.dot(layer_output_2, weights_3) + biases_3)
    return np.argmax(predictions, axis=1)

In [7]:
iterations = 10
learning_rate = 0.01
weights_1, biases_1, weights_2, biases_2, weights_3, biases_3 = weights_back_pro(X_train, y_train, iterations, learning_rate)

y_pred = predict(X_train, weights_1, biases_1, weights_2, biases_2, weights_3, biases_3)

# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

iteration0: Loss: 2.3150857376972462
iteration1: Loss: 2.2981400105523226
iteration2: Loss: 2.2908247628047755
iteration3: Loss: 2.284950451480555
iteration4: Loss: 2.278887219905517
iteration5: Loss: 2.2720040228712652
iteration6: Loss: 2.2639002405677906
iteration7: Loss: 2.254257853986804
iteration8: Loss: 2.2427499745277157
iteration9: Loss: 2.229160006719492
Test Accuracy: 18.98%


In [20]:
model = Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics= ['categorical_crossentropy', 'accuracy']
)


y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)

model.fit(X_train, y_train_one_hot, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1012 - categorical_crossentropy: 23.1250 - loss: 23.1250 - val_accuracy: 0.1021 - val_categorical_crossentropy: 2.3030 - val_loss: 2.3030
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0978 - categorical_crossentropy: 2.3029 - loss: 2.3029 - val_accuracy: 0.0952 - val_categorical_crossentropy: 2.3030 - val_loss: 2.3030
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1001 - categorical_crossentropy: 2.3026 - loss: 2.3026 - val_accuracy: 0.0997 - val_categorical_crossentropy: 2.3030 - val_loss: 2.3030
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0987 - categorical_crossentropy: 2.3025 - loss: 2.3025 - val_accuracy: 0.0952 - val_categorical_crossentropy: 2.3031 - val_loss: 2.3031
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1016 - categorical_crossentropy: 2.3026 - loss: 2.3026 - val_accuracy: 0.0997 - val_categorical_crossentropy: 2.3027 - val_l

In [18]:
model.evaluate(X_test, y_test_one_hot)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1006 - categorical_crossentropy: 2.3026 - loss: 2.3026


[2.302616596221924, 2.302616596221924, 0.10000000149011612]

In [19]:
X_test.shape

(10000, 3072)